In [12]:
import plotly as py
import plotly.graph_objs as go
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import glob

# Select signal ids
sig_ids = np.array([123,124,125,162,163,164,408,409,410,435,436,437])

# Load selected signals
dir_list = [f'../preprocessed_data/pp_train_db20/pp_train_db20_{num:d}.npy' for num in range(30)]
pp_signals = np.vstack([np.load(p) for p in dir_list])

pp_signals = pp_signals[sig_ids]

# Load metadata as well
meta_train, meta_test = pd.read_csv('../data/metadata_train.csv'), pd.read_csv('../data/metadata_test.csv')

# Load oof preds
oof_preds = pd.read_hdf('../level_1_preds/lgbm_v23_0.6609_pp_oof.h5').values[:, 0]

# Get target
target = meta_train['target'].values

In [13]:
# Add to data to plot
data = []
button_list = []

for i, (sig_id, pp_sig) in enumerate(zip(sig_ids, pp_signals)):
    
    # Build trace
    data.append(
        go.Scatter(
            x = np.arange(0, pp_sig.size, 1),
            y = pp_sig,
            visible = False
        )
    )
    
    # To hide all plots except current one
    visibility_array = np.zeros(pp_signals.shape[0])
    visibility_array[i] = 1
    visibility_array = list(visibility_array.astype(np.bool))
    
    # Build dropdown menu entry
    button_list.append(
        {
            'label' : f'Sig {sig_id:d} - Tgt {target[sig_id]:d}',
            'method' : 'update',
            'args' : [
                {'visible': visibility_array},
                {'title': f'Signal number {sig_id:d} - Target {target[sig_id]:d}',}]
        }
    )

    
    
py.offline.plot(
    {
        "data": data, 
        "layout": go.Layout(
            title="Line Chart",
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='rgb(0,0,0)'
            ),
            updatemenus=[
                dict(
                    active=-1,
                    buttons=button_list,
                )
            ],
        )
    },
    filename='single_signal.html',
)

'file:///Users/miguelrita/Documents/Kaggle/remote_three/notebooks/single_signal.html'